# Test of GraphSAGE
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [1]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:0


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [5]:
datasets = ['train', 'valid', 'test']
# datasets = ['valid']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp3/training_data/transR_50/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data/transR_50/train.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data/transR_50/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data/transR_50/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(16)

- Turn the print message to a log file

In [8]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_transR_50.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/1026_13:40_GraphSAGE_transR_50.log


### Model

In [9]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
        h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [29]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes):
        super().__init__()
        self.W = nn.Linear(out_feats*2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        
        print(h_u.shape)
        print(h_v.shape)
        
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [30]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes)
      
    def forward(self, g, node_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h)
        
        return logits

- Model Forward  

In [13]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
    logits = model(batched_g, batched_g.ndata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 5000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [14]:
seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [15]:
# model.layer1.fc_self.weight
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

- Check if model really load the model_dict

In [16]:
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

In [22]:
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]

In [32]:
import csv
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt"
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
criterion = nn.CrossEntropyLoss()

kf = KFold(n_splits=5, shuffle=False, random_state=None)
train_data = [item for item in dataloaders['train']]
kf_splits = kf.split(train_data)

total_steps = 20

# save the best model
best_val_loss = float('inf')
patience = 4  # Number of epochs with no improvement after which training will be stopped.

for fold, (train_index, val_index) in tqdm(enumerate(kf_splits)):
    train_set = [train_data[i] for i in train_index]
    val_set = [train_data[i] for i in val_index]
    
    # 初始化模型和优化器
    model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
    
    waiting = 0  # The number of epochs with no improvement so far.
    early_stop = False

    for epoch in tqdm(range(total_steps)):
        # Train
        model.train()
        total_loss = 0.0
        total_accuracy = 0.0
        num_batches = 0

        for batched_g in tqdm(train_set, desc="Training", position=0, leave=True):
            num_batches += 1
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_accuracy += accuracy.item()


        add_log_msg(f"total batches: {num_batches}")

        avg_loss = total_loss / num_batches
        avg_accuracy = total_accuracy / num_batches

        add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')


        # Validation Part
        model.eval()
        total_accuracy = 0.0
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batched_g in tqdm(val_set, desc="Validation", position=0, leave=True):
                loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
                total_accuracy += accuracy.item()
                total_loss += loss.item()
                num_batches += 1

        avg_accuracy = total_accuracy / num_batches
        current_loss = total_loss / num_batches

        add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')


        if current_loss < best_val_loss:
            best_val_loss = current_loss
            waiting = 0

            if os.path.exists(best_model_path):
                os.remove(best_model_path)
                add_log_msg("Find a better model!!")

            torch.save(model.state_dict(), best_model_path)

        else:
            waiting += 1
            if waiting >= patience:
                early_stop = True

        if epoch == total_steps - 1 or early_stop == True:  # Last epoch
            pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt'
            model.load_state_dict(torch.load(pretrained_model_path))
            model.to(device)

            true_labels = []
            predicted_labels = []

            with torch.no_grad():
                for batched_g in tqdm(train_set, desc="Generating Train Report", position=0, leave=True):
                    loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                    labels = batched_g.edata['label'].to(device)

                    true_labels.extend(labels.cpu().numpy())
                    predicted_labels.extend(predicted.cpu().numpy())

            # Assuming that the mapping and other things are defined earlier or globally
            training_mapped_true_labels = [label_mapping[label] for label in true_labels]
            training_mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

            # Generate classification report
            train_report = classification_report(training_mapped_true_labels, training_mapped_predicted_labels)
            add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

        if early_stop:
            add_log_msg("============================== Early stopping ==================================")
            break


0it [00:00, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/6600 [00:00<?, ?it/s]

torch.Size([164, 2048])
torch.Size([164, 2048])
torch.Size([292, 2048])
torch.Size([292, 2048])
torch.Size([67, 2048])
torch.Size([67, 2048])
torch.Size([113, 2048])
torch.Size([113, 2048])
torch.Size([91, 2048])
torch.Size([91, 2048])
torch.Size([995, 2048])
torch.Size([995, 2048])
torch.Size([127, 2048])
torch.Size([127, 2048])
torch.Size([2064, 2048])
torch.Size([2064, 2048])
torch.Size([89, 2048])
torch.Size([89, 2048])
torch.Size([203, 2048])
torch.Size([203, 2048])
torch.Size([71, 2048])
torch.Size([71, 2048])
torch.Size([451, 2048])
torch.Size([451, 2048])
torch.Size([2165, 2048])
torch.Size([2165, 2048])
torch.Size([288, 2048])
torch.Size([288, 2048])
torch.Size([223, 2048])
torch.Size([223, 2048])
torch.Size([362, 2048])
torch.Size([362, 2048])
torch.Size([167, 2048])
torch.Size([167, 2048])
torch.Size([235, 2048])
torch.Size([235, 2048])
torch.Size([467, 2048])
torch.Size([467, 2048])
torch.Size([68, 2048])
torch.Size([68, 2048])
torch.Size([274, 2048])
torch.Size([274, 2048]

torch.Size([93, 2048])
torch.Size([93, 2048])
torch.Size([525, 2048])
torch.Size([525, 2048])
torch.Size([48, 2048])
torch.Size([48, 2048])
torch.Size([377, 2048])
torch.Size([377, 2048])
torch.Size([203, 2048])
torch.Size([203, 2048])
torch.Size([303, 2048])
torch.Size([303, 2048])
torch.Size([115, 2048])
torch.Size([115, 2048])
torch.Size([86, 2048])
torch.Size([86, 2048])
torch.Size([81, 2048])
torch.Size([81, 2048])
torch.Size([356, 2048])
torch.Size([356, 2048])
torch.Size([317, 2048])
torch.Size([317, 2048])
torch.Size([124, 2048])
torch.Size([124, 2048])
torch.Size([1668, 2048])
torch.Size([1668, 2048])
torch.Size([187, 2048])
torch.Size([187, 2048])
torch.Size([417, 2048])
torch.Size([417, 2048])
torch.Size([211, 2048])
torch.Size([211, 2048])
torch.Size([449, 2048])
torch.Size([449, 2048])
torch.Size([90, 2048])
torch.Size([90, 2048])
torch.Size([61, 2048])
torch.Size([61, 2048])
torch.Size([320, 2048])
torch.Size([320, 2048])
torch.Size([468, 2048])
torch.Size([468, 2048])
to

...

torch.Size([151, 2048])
torch.Size([151, 2048])
torch.Size([96, 2048])
torch.Size([96, 2048])
torch.Size([299, 2048])
torch.Size([299, 2048])
torch.Size([170, 2048])
torch.Size([170, 2048])
torch.Size([122, 2048])
torch.Size([122, 2048])
torch.Size([229, 2048])
torch.Size([229, 2048])
torch.Size([68, 2048])
torch.Size([68, 2048])
torch.Size([103, 2048])
torch.Size([103, 2048])
torch.Size([101, 2048])
torch.Size([101, 2048])
torch.Size([420, 2048])
torch.Size([420, 2048])
torch.Size([333, 2048])
torch.Size([333, 2048])
torch.Size([463, 2048])
torch.Size([463, 2048])
torch.Size([148, 2048])
torch.Size([148, 2048])
torch.Size([77, 2048])
torch.Size([77, 2048])
torch.Size([123, 2048])
torch.Size([123, 2048])
torch.Size([1823, 2048])
torch.Size([1823, 2048])
torch.Size([140, 2048])
torch.Size([140, 2048])
torch.Size([84, 2048])
torch.Size([84, 2048])
torch.Size([130, 2048])
torch.Size([130, 2048])
torch.Size([216, 2048])
torch.Size([216, 2048])
torch.Size([86, 2048])
torch.Size([86, 2048])


torch.Size([327, 2048])
torch.Size([327, 2048])
torch.Size([148, 2048])
torch.Size([148, 2048])
torch.Size([46, 2048])
torch.Size([46, 2048])
torch.Size([76, 2048])
torch.Size([76, 2048])
torch.Size([248, 2048])
torch.Size([248, 2048])
torch.Size([60, 2048])
torch.Size([60, 2048])
torch.Size([91, 2048])
torch.Size([91, 2048])
torch.Size([254, 2048])
torch.Size([254, 2048])
torch.Size([112, 2048])
torch.Size([112, 2048])
torch.Size([142, 2048])
torch.Size([142, 2048])
torch.Size([2026, 2048])
torch.Size([2026, 2048])
torch.Size([207, 2048])
torch.Size([207, 2048])
torch.Size([310, 2048])
torch.Size([310, 2048])
torch.Size([276, 2048])
torch.Size([276, 2048])
torch.Size([71, 2048])
torch.Size([71, 2048])
torch.Size([267, 2048])
torch.Size([267, 2048])
torch.Size([126, 2048])
torch.Size([126, 2048])
torch.Size([265, 2048])
torch.Size([265, 2048])
torch.Size([72, 2048])
torch.Size([72, 2048])
torch.Size([77, 2048])
torch.Size([77, 2048])
torch.Size([214, 2048])
torch.Size([214, 2048])
torc

torch.Size([991, 2048])
torch.Size([991, 2048])
torch.Size([82, 2048])
torch.Size([82, 2048])
torch.Size([211, 2048])
torch.Size([211, 2048])
torch.Size([177, 2048])
torch.Size([177, 2048])
torch.Size([741, 2048])
torch.Size([741, 2048])
torch.Size([1641, 2048])
torch.Size([1641, 2048])
torch.Size([406, 2048])
torch.Size([406, 2048])
torch.Size([163, 2048])
torch.Size([163, 2048])
torch.Size([146, 2048])
torch.Size([146, 2048])
torch.Size([96, 2048])
torch.Size([96, 2048])
torch.Size([201, 2048])
torch.Size([201, 2048])
torch.Size([93, 2048])
torch.Size([93, 2048])
torch.Size([1195, 2048])
torch.Size([1195, 2048])
torch.Size([67, 2048])
torch.Size([67, 2048])
torch.Size([166, 2048])
torch.Size([166, 2048])
torch.Size([97, 2048])
torch.Size([97, 2048])
torch.Size([484, 2048])
torch.Size([484, 2048])
torch.Size([180, 2048])
torch.Size([180, 2048])
torch.Size([104, 2048])
torch.Size([104, 2048])
torch.Size([309, 2048])
torch.Size([309, 2048])
torch.Size([363, 2048])
torch.Size([363, 2048]

torch.Size([43, 2048])
torch.Size([43, 2048])
torch.Size([289, 2048])
torch.Size([289, 2048])
torch.Size([209, 2048])
torch.Size([209, 2048])
torch.Size([129, 2048])
torch.Size([129, 2048])
torch.Size([58, 2048])
torch.Size([58, 2048])
torch.Size([139, 2048])
torch.Size([139, 2048])
torch.Size([915, 2048])
torch.Size([915, 2048])
torch.Size([236, 2048])
torch.Size([236, 2048])
torch.Size([75, 2048])
torch.Size([75, 2048])
torch.Size([388, 2048])
torch.Size([388, 2048])
torch.Size([208, 2048])
torch.Size([208, 2048])
torch.Size([153, 2048])
torch.Size([153, 2048])
torch.Size([130, 2048])
torch.Size([130, 2048])
torch.Size([42, 2048])
torch.Size([42, 2048])
torch.Size([197, 2048])
torch.Size([197, 2048])
torch.Size([233, 2048])
torch.Size([233, 2048])
torch.Size([269, 2048])
torch.Size([269, 2048])
torch.Size([80, 2048])
torch.Size([80, 2048])
torch.Size([66, 2048])
torch.Size([66, 2048])
torch.Size([649, 2048])
torch.Size([649, 2048])
torch.Size([86, 2048])
torch.Size([86, 2048])
torch.

KeyboardInterrupt: 

In [28]:
import csv
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt"
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
criterion = nn.CrossEntropyLoss()

kf = KFold(n_splits=5, shuffle=False, random_state=None)
train_data = [item for item in dataloaders['train']]
kf_splits = kf.split(train_data)

total_steps = 20

# save the best model
best_val_loss = float('inf')
patience = 4  # Number of epochs with no improvement after which training will be stopped.

for fold, (train_index, val_index) in tqdm(enumerate(kf_splits)):
    train_set = [train_data[i] for i in train_index]
    val_set = [train_data[i] for i in val_index]
    
    # 初始化模型和优化器
    model = Model(in_features=50, hidden_features=1024, out_features=2048, num_classes=167)
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)
    
    waiting = 0  # The number of epochs with no improvement so far.
    early_stop = False

    for epoch in tqdm(range(total_steps)):
        # Train
        model.train()
        total_loss = 0.0
        total_accuracy = 0.0
        num_batches = 0

        for batched_g in tqdm(train_set, desc="Training", position=0, leave=True):
            num_batches += 1
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_accuracy += accuracy.item()


        add_log_msg(f"total batches: {num_batches}")

        avg_loss = total_loss / num_batches
        avg_accuracy = total_accuracy / num_batches

        add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')


        # Validation Part
        model.eval()
        total_accuracy = 0.0
        total_loss = 0.0
        num_batches = 0

        with torch.no_grad():
            for batched_g in tqdm(val_set, desc="Validation", position=0, leave=True):
                loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
                total_accuracy += accuracy.item()
                total_loss += loss.item()
                num_batches += 1

        avg_accuracy = total_accuracy / num_batches
        current_loss = total_loss / num_batches

        add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')


        if current_loss < best_val_loss:
            best_val_loss = current_loss
            waiting = 0

            if os.path.exists(best_model_path):
                os.remove(best_model_path)
                add_log_msg("Find a better model!!")

            torch.save(model.state_dict(), best_model_path)

        else:
            waiting += 1
            if waiting >= patience:
                early_stop = True

        if epoch == total_steps - 1 or early_stop == True:  # Last epoch
            pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt'
            model.load_state_dict(torch.load(pretrained_model_path))
            model.to(device)

            true_labels = []
            predicted_labels = []

            with torch.no_grad():
                for batched_g in tqdm(train_set, desc="Generating Train Report", position=0, leave=True):
                    loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                    labels = batched_g.edata['label'].to(device)

                    true_labels.extend(labels.cpu().numpy())
                    predicted_labels.extend(predicted.cpu().numpy())

            # Assuming that the mapping and other things are defined earlier or globally
            training_mapped_true_labels = [label_mapping[label] for label in true_labels]
            training_mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

            # Generate classification report
            train_report = classification_report(training_mapped_true_labels, training_mapped_predicted_labels)
            add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

        if early_stop:
            add_log_msg("============================== Early stopping ==================================")
            break


0it [00:00, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Training:   0%|          | 0/6600 [00:00<?, ?it/s]

10/26/2023, 15:42:40# total batches: 6600
10/26/2023, 15:42:40# Epoch 0 | Train Loss: 0.5085 | Train Accuracy: 0.8903


Validation:   0%|          | 0/1650 [00:00<?, ?it/s]

...

10/26/2023, 16:35:58# labels of Validation: tensor([ 95,  95,  95,  50,  50,  50, 154, 154, 154, 154, 154, 154, 154, 154,
        154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154,
        154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154, 154,
        154, 154, 154, 154, 154, 154,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,
         65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65,  65, 154, 154,
        154, 154, 15

Generating Train Report:   0%|          | 0/6600 [00:00<?, ?it/s]

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/26/2023, 16:37:55# Train Classification Report at Epoch 3:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00     41408
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00      3918
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00      3265
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00      9750
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       661
    T1003.003_f049b89533298c2d6cd37a940248b219       0.00      0.00      0.00       649
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00      3852
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00      3900
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.00      0.00      0.00       643
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.15

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt'
model.load_state_dict(torch.load(pretrained_model_path))
model.to(device)

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['train'], desc="Generating Train Report", position=0, leave=True):
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
        labels = batched_g.edata['label'].to(device)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Assuming that the mapping and other things are defined earlier or globally
training_mapped_true_labels = [label_mapping[label] for label in true_labels]
training_mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# Generate classification report
train_report = classification_report(training_mapped_true_labels, training_mapped_predicted_labels)
add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

Generating Train Report:   0%|          | 0/8250 [00:00<?, ?it/s]

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/26/2023, 15:27:03# Train Classification Report at Epoch 12:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00     51200
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00      4800
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00      4000
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00     12000
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.00      0.00      0.00       800
    T1003.003_f049b89533298c2d6cd37a940248b219       0.00      0.00      0.00       800
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00      4800
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00      4800
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.02      0.02      0.02       800
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.1

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [ ]:
# load the pretrained model
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50-9.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")